## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.

## 목적:
- LangChain과 Qdrant를 사용하여 벡터 데이터를 int8 양자화하여 저장 및 검색
- Qdrant에 텍스트 데이터를 벡터로 변환하고 int8로 양자화하여 저장
- 사용자가 입력한 텍스트와 유사한 데이터를 검색

## 왜 필요한가?
- 벡터 기반 검색은 문서 검색, 챗봇, 추천 시스템 등에 필수적임
- Qdrant는 고성능 벡터 검색을 지원하는 데이터베이스로, 양자화를 통해 저장 공간과 검색 속도를 최적화할 수 있음

## 주요 개념:
- **LangChain**: 자연어 처리(NLP)와 벡터 검색을 쉽게 결합할 수 있는 라이브러리
- **Qdrant**: 밀집 벡터를 저장하고 빠르게 검색할 수 있는 오픈소스 벡터 데이터베이스
- **벡터 양자화**: 벡터의 요소를 int8과 같은 저비트 데이터 타입으로 변환하여 저장 공간을 절약하고 검색 효율을 높이는 방법

In [2]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
import os
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, ScalarQuantization, ScalarQuantizationConfig, ScalarType
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
# Qdrant 서버 연결
qdrant_client = QdrantClient(host="localhost", port=6333)

In [4]:
collection_name = "ex10_quantized_vectors"

# Qdrant 컬렉션 생성 (없을 경우 자동 생성)
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=384,  # 벡터 차원 수 (임베딩 모델과 일치해야 함)
            distance=Distance.COSINE,  # 유사도 계산 방식 (코사인 거리 사용)
            on_disk=True  # 원본 벡터를 디스크에 저장하여 유지
        ),
        quantization_config=ScalarQuantization(
            scalar=ScalarQuantizationConfig(
                type=ScalarType.INT8,  # int8 타입으로 양자화
                always_ram=True  # 양자화된 벡터를 RAM에 유지하여 빠른 검색 지원
            )
        )
    )
    print(f"Qdrant 컬렉션 '{collection_name}' 생성 완료!")

Qdrant 컬렉션 'ex10_quantized_vectors' 생성 완료!


In [5]:
# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [6]:
# Qdrant 기반 벡터 저장소 생성
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=embeddings
)

In [7]:
# 샘플 텍스트 및 메타데이터
texts = [
    "Hello World",
    "LangChain is great!",
    "Vector databases are useful",
    "Qdrant is a fast vector DB"
]
metadata = [{"source": f"text{i+1}"} for i in range(len(texts))]

# 텍스트를 벡터로 변환 후 저장
vectorstore.add_texts(texts, metadatas=metadata)
print("Qdrant에 벡터 저장 완료!")

Qdrant에 벡터 저장 완료!


In [8]:
# 검색을 위한 쿼리 텍스트 설정
query_text = "LangChain"

# 유사한 벡터 검색 실행 (상위 2개 검색)
search_results = vectorstore.similarity_search(query_text, k=2)

# 검색 결과 출력
print(f"'{query_text}' 검색 결과:")
for i, doc in enumerate(search_results):
    print(f"{i+1}. 문서 내용: {doc.page_content}, 메타데이터: {doc.metadata}")


'LangChain' 검색 결과:
1. 문서 내용: LangChain is great!, 메타데이터: {'source': 'text2', '_id': 'e12dc91c-658a-4ee8-807f-ee5a8e3cb4ff', '_collection_name': 'ex10_quantized_vectors'}
2. 문서 내용: Hello World, 메타데이터: {'source': 'text1', '_id': '781a5fa4-317a-4110-9831-4157d5bc4713', '_collection_name': 'ex10_quantized_vectors'}
